In [1]:
# Importando Bibliotecas
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

In [2]:
# Lendo dados
df = pd.read_csv(
    'Rotatividade_Clientes_Telco.csv', 
    sep=','
)
# Informações importantes dos dados: types, non-nulls e colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID_Consumidor                 7043 non-null   object 
 1   Genero                        7043 non-null   object 
 2   Idoso                         7043 non-null   int64  
 3   Parceiro                      7043 non-null   object 
 4   Dependentes                   7043 non-null   object 
 5   Nro_Meses_Cliente_na_Empresa  7043 non-null   int64  
 6   Servico_Telefone              7043 non-null   object 
 7   Muitas_Linhas_Telefone        7043 non-null   object 
 8   Servico_Internet              7043 non-null   object 
 9   Seguranca_OnLine              7043 non-null   object 
 10  Backup_OnLine                 7043 non-null   object 
 11  Dispositivo_Protecao          7043 non-null   object 
 12  Suporte_Tecnico               7043 non-null   object 
 13  Str

In [3]:
# Drop coluna ID_Consumidor -- não será usada no modelo
df = df.drop(columns=['ID_Consumidor'])
df.tail()

,Genero,Idoso,Parceiro,Dependentes,Nro_Meses_Cliente_na_Empresa,Servico_Telefone,Muitas_Linhas_Telefone,Servico_Internet,Seguranca_OnLine,Backup_OnLine,Dispositivo_Protecao,Suporte_Tecnico,Streaming_TV,Streaming_Filmes,Contrato,Faturamento_Sem_Papel,Metodo_Pagamento,Valor_Mensal,Valor_Total,Rotatividade
7038,M,0,S,S,24,S,S,DSL,S,N,S,S,S,S,Anual,S,Cheque_Fisico,84.80,1990.50,N
7039,F,0,S,S,72,S,S,Fibra_otica,N,S,S,N,S,S,Anual,S,Cartao_Credito,103.20,7362.90,N
7040,F,0,S,S,11,N,Sem_Telefone,DSL,S,N,N,N,N,N,Mes-a-mes,S,Cheque_Eletronico,29.60,346.45,N
7041,M,1,S,N,4,S,S,Fibra_otica,N,N,N,N,N,N,Mes-a-mes,S,Cheque_Fisico,74.40,306.60,S
7042,M,0,N,N,66,S,N,Fibra_otica,S,N,S,S,S,S,Bi-Anual,S,Transferencia_bancaria,105.65,6844.50,N


In [4]:
# Conferindo se há valores nulos no DataFrame
df.isnull().sum()

Genero                          0
Idoso                           0
Parceiro                        0
Dependentes                     0
Nro_Meses_Cliente_na_Empresa    0
Servico_Telefone                0
Muitas_Linhas_Telefone          0
Servico_Internet                0
Seguranca_OnLine                0
Backup_OnLine                   0
Dispositivo_Protecao            0
Suporte_Tecnico                 0
Streaming_TV                    0
Streaming_Filmes                0
Contrato                        0
Faturamento_Sem_Papel           0
Metodo_Pagamento                0
Valor_Mensal                    0
Valor_Total                     0
Rotatividade                    0
dtype: int64

In [5]:
# Criando função para transformar dados categoricos em numéricos para ler no modelo
def to_numeric(coluna):
    if coluna.dtype=='object':
        coluna = LabelEncoder().fit_transform(coluna)
    return coluna

In [6]:
# Aplicando função
df_numeric = df.apply(lambda x: to_numeric(x))
df_numeric.head()

,Genero,Idoso,Parceiro,Dependentes,Nro_Meses_Cliente_na_Empresa,Servico_Telefone,Muitas_Linhas_Telefone,Servico_Internet,Seguranca_OnLine,Backup_OnLine,Dispositivo_Protecao,Suporte_Tecnico,Streaming_TV,Streaming_Filmes,Contrato,Faturamento_Sem_Papel,Metodo_Pagamento,Valor_Mensal,Valor_Total,Rotatividade
0,0,0,1,0,1,0,2,0,0,1,0,0,0,0,2,1,1,29.85,29.85,0
1,1,0,0,0,34,1,0,0,1,0,1,0,0,0,0,0,2,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,1,1,0,0,0,0,2,1,2,53.85,108.15,1
3,1,0,0,0,45,0,2,0,1,0,1,1,0,0,0,0,3,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,2,1,1,70.70,151.65,1


In [7]:
# Separando dados em dados de treino e teste
X = df_numeric.drop(columns = ['Rotatividade'])
y = df_numeric['Rotatividade'].values

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = 0.30, 
    random_state = 40, 
    stratify=y
)

In [14]:
# Teste de acuracidade com vários tamanhos de estimadores
accuracy = 0
n_estimators = None
for x in range(100, 600, 100):
    # Realizando análise preditiva
    RandomForest = RandomForestClassifier(
        n_estimators=x,
        n_jobs=-1,
        random_state=888,
    )
    RandomForest.fit(X_train, y_train)
    prediction = RandomForest.predict(X_test)

    test_accuracy = metrics.accuracy_score(y_test, prediction)
    if accuracy < test_accuracy:
        accuracy = test_accuracy
        n_estimators = x
        
    print(f'O modelo com {x} arvores apresentou %.2f%% de acuracidade' % (accuracy * 100))
print('---------')
print(f'O melhor modelo é com {n_estimators} arvores.')
    

O modelo com 100 arvores apresentou 78.23% de acuracidade
O modelo com 200 arvores apresentou 78.32% de acuracidade
O modelo com 300 arvores apresentou 78.89% de acuracidade
O modelo com 400 arvores apresentou 78.89% de acuracidade
O modelo com 500 arvores apresentou 78.89% de acuracidade
---------
O melhor modelo é com 300 arvores.


In [15]:
RandomForest = RandomForestClassifier(
    n_estimators=n_estimators,
    n_jobs=-1,
    random_state=888,
)
RandomForest.fit(X_train, y_train)
prediction = RandomForest.predict(X_test)

accuracy = metrics.accuracy_score(y_test, prediction)

In [16]:
# Avaliando importancias das colunas para a decisão do modelo
feature_importances = RandomForest.feature_importances_

# Gerando DataFrame e classificando
importancias = {
    'Coluna':[x for x in X.columns.values],
    'Importancia':[round(x*100, 2) for x in feature_importances]
}

importancias = pd.DataFrame(importancias).sort_values('Importancia', ascending=False).reset_index(drop=True)
importancias

,Coluna,Importancia
0,Valor_Total,18.81
1,Valor_Mensal,17.32
2,Nro_Meses_Cliente_na_Empresa,15.71
3,Contrato,8.08
4,Metodo_Pagamento,5.07
5,Seguranca_OnLine,4.73
6,Suporte_Tecnico,3.96
7,Genero,2.89
8,Backup_OnLine,2.71
9,Servico_Internet,2.56
